Se obtienen datos de la cotización de Facebook (META)

In [ ]:
#carga de librerías
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from io import StringIO

In [ ]:
# Los datos proporcionados
data_str = """Años,Facebook (al mes),WhatsApp (al año),Instagram (total)
2008,154,,
2009,360,,
2010,608,,
2011,845,,
2012,1056,103,
2013,1228,213,150
2014,1393,409,300
2015,1591,719,435
2016,1860,1076,600
2017,2129,1323,885
2018,2320,1560,1115
2019,2498,1813,1300
2020,2797,2102,1755
2021,2910,2289,2145
2022,2963,2413,2350
2023,3021,2413,2365
"""

# Crear un DataFrame de Pandas
df = pd.read_csv(StringIO(data_str))

# Crear un gráfico de líneas con Plotly
fig1 = go.Figure()

# Añadir trazas para cada columna
for col in df.columns[1:]:  # Excluir la columna 'Años'
    fig1.add_trace(go.Scatter(x=df['Años'], y=df[col], mode='lines', name=col))

# Configurar el diseño del gráfico
fig1.update_layout(
    title='Usuarios de Redes Sociales a lo largo del tiempo',
    xaxis_title='Años',
    yaxis_title='Número de Usuarios',
    legend=dict(title='Redes Sociales'),
    template='plotly_dark'  # Fondo oscuro
)

# Mostrar el gráfico
fig1.show()

In [ ]:
#Obtención de datos

db = yf.download('META', start = '2010-07-01', end = '2023-12-05')
db


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.189480,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,33.993931,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,30.967144,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,31.966084,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,32.994991,50237200
...,...,...,...,...,...,...
2023-11-28,333.399994,339.380005,333.399994,338.989990,338.630707,12637200
2023-11-29,339.690002,339.899994,330.779999,332.200012,331.847931,16024500
2023-11-30,331.890015,333.500000,322.399994,327.149994,326.803253,23146400


In [ ]:
# Media movil exponencial a 9 periodos
db['EXMM9'] = db['Adj Close'].ewm(span = 9, adjust = False).mean()

 # Media movil simple a 20 dias
db['MM20'] = db['Adj Close'].rolling(window=20).mean()

#Media Móvil Simple de 50 Periodos
db['MM50'] = db['Adj Close'].rolling(window=50).mean()

#Media Móvil Simple de 100 Periodos
db['MM100'] = db['Adj Close'].rolling(window=100).mean()

#Media Móvil Simple de 200 Periodos
db['MM200'] = db['Adj Close'].rolling(window=200).mean()

In [ ]:
# Bandas de Bollinger con MM 20 dias
 # Banda superior
db['BB_UPPER'] = db['MM20'] + 2*db['Adj Close'].rolling(window=20).std()
#Banda inferior
db['BB_LOWER'] = db['MM20'] - 2*db['Adj Close'].rolling(window=20).std()

In [ ]:
# Rango medio verdadero a 14 periodos
db['TR'] = pd.DataFrame(np.maximum(np.maximum(db['High'] - db['Low'], abs(db['High'] - db['Adj Close'].shift())), abs(db['Low'] - db['Adj Close'].shift())), index = db.index)

#Se crea colummna atr en el dataframe
db['ATR'] = db['TR'].rolling(window = 14).mean()

In [ ]:
# Se agrega RSI para 14 periodos

#Se calcula delta
delta = db['Adj Close'].diff()
#Valores de ganancia
gain = delta.where(delta > 0,0)
#Valores de perdida
loss = -delta.where(delta < 0,0)
#Medición del valor de ganancia promedio de 14 períodos
avg_gain = gain.rolling(window=14).mean()

# Medición del valor de pérdida promedio de 14 períodos
avg_loss = loss.rolling(window=14).mean()
#Se calcula rs
rs = avg_gain/avg_loss
#Se crea la columna rsi
db['RSI'] = 100 - (100 / (1 + rs))

In [ ]:
# Se trazan los graficos de velas
fig2 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,row_heights=[0.6, 0.10, 0.10])


# Candlestick
fig2.add_trace(go.Candlestick(x=db.index,
                             open=db['Open'],
                             high=db['High'],
                             low=db['Low'],
                             close=db['Adj Close'],
                             name='META'),
              row=1, col=1)



# Bandas de Bollinger
fig2.add_trace(go.Scatter(x=db.index,
                         y=db['BB_UPPER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Banda superior'),
              row=1, col=1)

fig2.add_trace(go.Scatter(x=db.index,
                         y=db['BB_LOWER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Banda inferior'),
              row=1, col=1)

fig2.add_annotation(text='Facebook (META)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)


# Rango verdadero promedio (ATR)
fig2.add_trace(go.Scatter(x=db.index,
                         y=db['ATR'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='ATR'),
              row=2, col=1)


# Volumen
fig2.add_trace(go.Bar(x=db.index,
                     y=db['Volume'],
                     name='Volumen',
                     marker=dict(color='orange')),
              row=3, col=1)
# Layout
fig2.update_layout(title='META Grafico de velas desde el 1 de Julio, 2010 a 05 de diciembre,2023 ',
                  yaxis=dict(title='Precio (USD)'),
                  height=1000,
                 template = 'plotly_dark')

# Axes and subplots

fig2.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig2.update_xaxes(rangeslider_visible=False, row=3, col=1)
fig2.update_yaxes(title_text='ATR', row=2, col=1)
fig2.update_yaxes(title_text='Volumen', row=3, col=1)
fig2.update_yaxes(range=[0, 200000000], row=3, col=1)
fig2.show()


In [ ]:
# Se descargan los datos semanales de Meta
db2 = yf.download('META', start = '2010-07-01', end = '2023-12-05', interval='1wk')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
db2['EXMM9'] = db2['Adj Close'].ewm(span = 9, adjust = False).mean()

 # Media movil simple a 20 dias
db2['MM20'] = db2['Adj Close'].rolling(window=20).mean()

In [ ]:
# Crear un gráfico de violín para 'Adj Close'
fig3 = go.Figure()
fig3.add_trace(go.Violin(y=db2['Adj Close'], box_visible=True, line_color='#636EFA', meanline_visible=True, fillcolor='lightseagreen', opacity=0.6, name="Adj Close"))

# Añadir más gráficos de violín según sea necesario
fig3.add_trace(go.Violin(y=db2['Open'], box_visible=True, line_color='#EF553B', meanline_visible=True, fillcolor='lightcoral', opacity=0.6, name="Open"))
fig3.add_trace(go.Violin(y=db2['High'], box_visible=True, line_color='#00CC96', meanline_visible=True, fillcolor='lightgreen', opacity=0.6, name="High"))
fig3.add_trace(go.Violin(y=db2['Low'], box_visible=True, line_color='#AB63FA', meanline_visible=True, fillcolor='lightblue', opacity=0.6, name="Low"))

# Configurar botones de alternancia
fig3.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Cotización META 01/12/2021 a 05/12/2023"}]),
                dict(label="Adj Close",
                     method="update",
                     args=[{"visible": [True, False, False, False]},
                           {"title": "Cotización de cierre ajustado META 01/12/2021 a 05/12/2023"}]),
                dict(label="Open",
                     method="update",
                     args=[{"visible": [False, True, False, False]},
                           {"title": "Cotización apertura META 01/12/2021 a 05/12/2023"}]),
                dict(label="High",
                     method="update",
                     args=[{"visible": [False, False, True, False]},
                           {"title": "Cotización máxima META 01/12/2021 a 05/12/2023"}]),
                dict(label="Low",
                     method="update",
                     args=[{"visible": [False, False, False, True]},
                           {"title": "Cotización mínima META 01/12/2021 a 05/12/2023"}]),
            ]),
        )
    ])

# Ajustar el diseño del gráfico
fig3.update_layout(
    title='Cotización META 01/12/2021 a 05/12/2023',
    template='none'
)

# Mostrar el gráfico
fig3.show()

In [ ]:
# Definicion de parametros para el cruce de medias moviles
short_ma = 'EXMM9'
long_ma = 'MM20'

# Se crea un a señal que almacena las señales
db2['señal'] = 0

# Se generan las señales de entrada
for i in range(1, len(db2)):
    if db2[short_ma][i] > db2[long_ma][i] and db2[short_ma][i - 1] <= db2[long_ma][i - 1]:
# Señal de compra
        db2['señal'][i] = 1
    elif db2[short_ma][i] < db2[long_ma][i] and db2[short_ma][i - 1] >= db2[long_ma][i - 1]:
       # Señal de venta
        db2['señal'][i] = -1

# Calculando el retorno total
db2['retorno'] = db2['Adj Close'].pct_change()
db2['retorno_acumulado'] = (1 + db2['retorno']).cumprod() - 1

# Aplicacion de las señales a los retornos
db2['estrategia_retorno'] = db2['señal'].shift(1) * db2['retorno']

# Calculo del retorno acumulado
db2['estrategia_retorno_acumulativos'] = (1 + db2['estrategia_retorno']).cumprod() - 1

# Definicion del capital inicial
initial_capital = 100

# Calculo del valor de las accionespara capital inicial de 100
db2['valor_acciones'] = (1 + db2['estrategia_retorno']).cumprod() * initial_capital

# Se imprimen el numero de operaciones, capital inicial y capital final
num_trades = db2['señal'].abs().sum()
final_capital = db2['valor_acciones'].iloc[-1]
total_return = (final_capital - initial_capital) / initial_capital * 100

print('\n')
print(f"Cantidad de operaciones: {num_trades}")
print(f"Capital inicial: ${initial_capital}")
print(f"Capital final: ${final_capital:.2f}")
print(f"Retorno total: {total_return:.2f}%")
print('\n')

# Grafico del retorno total de la estrategia
fig4 = go.Figure()

fig4.add_trace(go.Scatter(x=db2.index,
                         y=db2['valor_acciones'].round(2),
                         mode='lines',
                         line=dict(color='#FFFF00'),
                         name='valor_acciones'))

fig4.update_layout(title='META-Estrategia de cruce de media móvil en datos semanales',
                  xaxis_title='Fecha',
                  yaxis_title='Valor ($)',
                  template='plotly_dark',
                 height = 600)

fig4.show()

<ipython-input-12-ac10dc528b51>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-ac10dc528b51>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-ac10dc528b51>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-ac10dc528b51>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/



Cantidad de operaciones: 21
Capital inicial: $100
Capital final: $112.81
Retorno total: 12.81%




In [ ]:
fig5 = go.Figure()

fig5.add_trace(go.Scatter(x=db.index,
                           y=db['RSI'],
                           mode='lines',
                           line=dict(color='#CBC3E3'),
                           name='RSI'))

# Configurar el diseño del fondo
fig5.update_layout(
    plot_bgcolor='black',  # Fondo negro
    paper_bgcolor='black',  # Fondo del área del gráfico negro
    font=dict(color='orange')  # Color del texto
)

# Configurar ejes
fig5.update_xaxes(color='white')
fig5.update_yaxes(color='white')

# Se agregan lineas de marca a 70 y 30
fig5.add_shape(type="line",
              x0=db.index[0], y0=70, x1=db.index[-1], y1=70,
              line=dict(color="red", width=2, dash="dot"),
              )
fig5.add_shape(type="line",
              x0=db.index[0], y0=30, x1=db.index[-1], y1=30,
              line=dict(color="#90EE90", width=2, dash="dot"),
              )

fig5.update_yaxes(title_text='RSI')

# Mostrar el gráfico
fig5.show()

In [ ]:
#necesario para la app en este entorno
from google.colab.output import eval_js

In [ ]:
!pip install dash
import dash
from dash import Dash, dcc, html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00


In [ ]:
#creación de la app
print(eval_js("google.colab.kernel.proxyPort(8034)"))

app = Dash()

colors = {'background': '#800080','text': 'black'}


# Definición del layout
app.layout = html.Div(children=[
    # Elementos en la parte superior de la página
    html.Div([
        html.H1(children='Análisis de estrategias de inversión. Empresa: Facebook (META)', style={'textAlign': 'center'}),

        #html.Div(children='''Evolucion de la cotización de META.''', style={'textAlign': 'center', 'color': 'black', "font-weight": "bold", 'fontSize': 20}),

    ]),

    # Pestañas
    dcc.Tabs(id='tabs', value='Presentacion', children=[
        # Pestaña "Presentación"
        dcc.Tab(id='Presentacion', label='Presentación', value='Presentacion', children=[
            dcc.Tab(id='Introducción', label='Introducción', value='Introducción',
                children=[ #crear la documentación del tablero
                    dcc.Markdown(
   '''
    En septiembre de 2006 se lanzó Facebook al público, de manera que cualquier persona mayor de 13 años podía acceder y dejaba de funcionar bajo invitación.
    La compañía muda su sede a Palo Alto y desde ahí orquesta su lanzamiento internacional, llegando a España en 2008.
    Con este crecimiento en usuarios, Facebook tarda un tiempo en salir a Bolsa pero cuando lo hace, en 2012, supera todas las expectativas: se convierte en la tercera mejor salida
    a los mercados de la historia en Estados Unidos y esto consolida a la compañía de Zuckerberg como una de las 5 grandes tecnológicas, junto con Google, Microsoft, Amazon y Apple.
    Según las cifras más recientes, hoy en día Instagram tiene 2.365 millones de cuentas de usuario y WhatsApp le supera con 2.413 millones de personas que abren la aplicación
    al menos una vez al año.
    Estas plataformas siguen funcionando de manera independiente, aunque han ido dando pasos hacia una creciente integración desde el punto de vista de los usuarios que también
    está orientada al mercado publicitario, el gran sostén económico de Meta desde hace años.
    En octubre de 2021 facebook Facebook cambió el nombre de su matriz a META desarrollo del metaverso. Esta arriesgada apuesta personal de Mark Zuckerberg ayudó a acercar público
    el concepto del metaverso, pero hizo perder miles de millones a Meta –se estima que las pérdidas son de 100.000 millones de dólares– y la confianza de los inversores, de manera
    que entre finales de 2021 y 2022, se dejó por el camino una parte importante de su valor en los mercados.
    Coincidiendo con la ralentización del sector tecnológico producida en 2022 tras el estallido de la Guerra de Ucrania y la bajada de la inversión en publicidad por el miedo
    a una recesión económica, el paso en falso con el metaverso le pasó factura a Meta y le llevó a despedir a 11.000 personas en noviembre de 2022 y a otras 10.000 en abril de 2023.
    Este trabajo persigue demostrar que decisiones desacertadas de la empresa META junto con el estancamiento en la cantidad de usuarios no la hacen una buena inversion a largo plazo,
    De este modo aconsejamos a los tenedores de acciones de esta empresa vender sus activos utilizando estrategias como el cruce de medias moviles utilizando ATR como indicador de volatilidad
''',style={'fontSize': 18, 'textAlign': 'justify'})
            ])
    ]),


        # Pestaña "Análisis Técnico"
        dcc.Tab(id='AnalisisTecnico', label='Evolución histórica', value='AnalisisTecnico', children=[
            html.Div([
                dcc.Graph(
                   id='graph1',
                   figure=fig1
                ),

                dcc.Graph(
                    id='graph2',
                    figure=fig2
                ),
                dcc.Graph(
                    id='graph3',
                    figure=fig3
                ),

            ])
        ]),


        # Pestaña "Estrategia"
        dcc.Tab(id='Estrategia', label='Estrategia', value='Estrategia', children=[
            html.Div([
                dcc.Graph(
                    id='graph4',
                    figure=fig4),
                dcc.Graph(
                    id='graph5',
                    figure=fig5
                ),
            ])
        ]),



        # Otras pestañas pueden ir aquí si lo necesitas
    ])
])

app.run_server(port=8034, mode='external', debug=False)

#una vez ejecutado hacer click en el link

https://3grgulpu7mx-496ff2e9c6d22116-8034-colab.googleusercontent.com/


<IPython.core.display.Javascript object>